In [ ]:
import torch
import torch.nn.functional as F

##### Example 1

In [ ]:
BATCH_SIZE = 4
SEQ_LEN = 8
HIDDEN_SIZE = 16

INTERMITTENT_SIZE = 16

In [ ]:
inputs = torch.randn(BATCH_SIZE, SEQ_LEN, HIDDEN_SIZE)

In [ ]:
weights = torch.randn(HIDDEN_SIZE, INTERMITTENT_SIZE)

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
inputs.shape, weights.shape

(torch.Size([4, 8, 16]), torch.Size([16, 16]))

In [ ]:
out = F.relu(torch.matmul(inputs, weights))

In [ ]:
out.shape

torch.Size([4, 8, 16])

Suppose that the `weights` parameter is very large and needs to be split across multiple GPUs to enable model parallelism.

Implement Megatron's approach to achieve this.

In [ ]:
def split_weights(x):
    n_cols = x.shape[-1]
    assert n_cols // 2 # can be ignored
    return x[:, :n_cols//2], x[:, n_cols//2:]

In [ ]:
weight1, weight2 = split_weights(weights)

In [ ]:
out1 = F.relu(torch.matmul(inputs, weight1)) # on devices 1

In [ ]:
out2 = F.relu(torch.matmul(inputs, weight2)) # on devices 2

Concat the output from all devices

In [ ]:
parallel_out = torch.cat([out1, out2], dim=2)

In [ ]:
parallel_out.shape

torch.Size([4, 8, 16])

In [ ]:
torch.eq(out, parallel_out)

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True

##### Example 2

In [ ]:
x = torch.randn(3, 5)

In [ ]:
q, k, v = 